In [ ]:
from google.colab import files

# Subo fichero
uploaded = files.upload()

# Ver qué nombre de fichero ha llegado
print("Ficheros subidos:", uploaded.keys())


Saving online_retail_II.xlsx to online_retail_II (1).xlsx
Ficheros subidos: dict_keys(['online_retail_II (1).xlsx'])


In [ ]:
import pandas as pd

# 1) Cargar el libro de Excel
xls = pd.ExcelFile("online_retail_II.xlsx")

# 2) Ver los nombres de las hojas
print("Hojas disponibles:", xls.sheet_names)

# 3) Cargar la primera hoja para ver una muestra
df_preview = xls.parse(xls.sheet_names[0])

print("Shape de la primera hoja:", df_preview.shape)
print(df_preview.head())


Hojas disponibles: ['Year 2009-2010', 'Year 2010-2011']
Shape de la primera hoja: (525461, 8)
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  


In [ ]:
import pandas as pd

# Volvemos a abrir el Excel
xls = pd.ExcelFile("online_retail_II.xlsx")

# Cargar cada hoja en un DataFrame
df_2009 = xls.parse('Year 2009-2010')
df_2010 = xls.parse('Year 2010-2011')

print("Shape 2009-2010:", df_2009.shape)
print("Shape 2010-2011:", df_2010.shape)


Shape 2009-2010: (525461, 8)
Shape 2010-2011: (541910, 8)


In [ ]:
# Añadimos columna 'period' para identificar el año de cada registro
df_2009["period"] = "2009-2010"
df_2010["period"] = "2010-2011"

# Unimos los dos DataFrames en uno solo
df_raw = pd.concat([df_2009, df_2010], ignore_index=True)

print("Shape total df_raw:", df_raw.shape)
print(df_raw.head())
print(df_raw["period"].value_counts())


Shape total df_raw: (1067371, 9)
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country     period  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  2009-2010  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  2009-2010  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  2009-2010  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  2009-2010  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  2009-2010  
period
2010-2011    541910
2009-2010    525461
Name: count, dtype: int64


In [ ]:
# Renombrar las columnas a snake_case
df_raw = df_raw.rename(columns={
    "Invoice": "invoice",
    "StockCode": "stock_code",
    "Description": "description",
    "Quantity": "quantity",
    "InvoiceDate": "invoice_date",
    "Price": "price",
    "Customer ID": "customer_id",
    "Country": "country"
})

print(df_raw.columns)


Index(['invoice', 'stock_code', 'description', 'quantity', 'invoice_date',
       'price', 'customer_id', 'country', 'period'],
      dtype='object')


In [ ]:
# 1) Variables de texto
df_raw["invoice"]    = df_raw["invoice"].astype(str).str.strip()
df_raw["stock_code"] = df_raw["stock_code"].astype(str).str.strip()
df_raw["description"] = df_raw["description"].astype(str).str.strip()
df_raw["country"]    = df_raw["country"].astype(str).str.strip()

# 2) Variables numéricas
df_raw["quantity"] = pd.to_numeric(df_raw["quantity"], errors="coerce")
df_raw["price"]    = pd.to_numeric(df_raw["price"], errors="coerce")
df_raw["customer_id"] = pd.to_numeric(df_raw["customer_id"], errors="coerce")

# 3) Fecha
df_raw["invoice_date"] = pd.to_datetime(df_raw["invoice_date"], errors="coerce")

# 4) Vista rápida del resultado
print(df_raw.dtypes)
print(df_raw.head())


invoice                 object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
price                  float64
customer_id            float64
country                 object
period                  object
dtype: object
  invoice stock_code                          description  quantity  \
0  489434      85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434     79323P                   PINK CHERRY LIGHTS        12   
2  489434     79323W                  WHITE CHERRY LIGHTS        12   
3  489434      22041          RECORD FRAME 7" SINGLE SIZE        48   
4  489434      21232       STRAWBERRY CERAMIC TRINKET BOX        24   

         invoice_date  price  customer_id         country     period  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  2009-2010  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  2009-2010  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  2009-201

In [ ]:
# Monetary (M): revenue = quantity * price
df_raw["revenue"] = df_raw["quantity"] * df_raw["price"]

print(df_raw[["quantity", "price", "revenue"]].head())
print("\nDescripción estadística de revenue:")
print(df_raw["revenue"].describe())


   quantity  price  revenue
0        12   6.95     83.4
1        12   6.75     81.0
2        12   6.75     81.0
3        48   2.10    100.8
4        24   1.25     30.0

Descripción estadística de revenue:
count    1.067371e+06
mean     1.806987e+01
std      2.924202e+02
min     -1.684696e+05
25%      3.750000e+00
50%      9.900000e+00
75%      1.770000e+01
max      1.684696e+05
Name: revenue, dtype: float64


In [ ]:
# Indicador de devolución / nota de crédito
df_raw["is_return"] = (
    df_raw["invoice"].str.startswith("C") |   # facturas que empiezan por 'C'
    (df_raw["quantity"] < 0)                  # o cantidad negativa
)

print(df_raw[["invoice", "quantity", "is_return"]].head(10))

print("\nDistribución de is_return (conteo):")
print(df_raw["is_return"].value_counts())

print("\nDistribución de is_return (porcentaje):")
print(df_raw["is_return"].value_counts(normalize=True) * 100)


  invoice  quantity  is_return
0  489434        12      False
1  489434        12      False
2  489434        12      False
3  489434        48      False
4  489434        24      False
5  489434        24      False
6  489434        24      False
7  489434        10      False
8  489435        12      False
9  489435        12      False

Distribución de is_return (conteo):
is_return
False    1044420
True       22951
Name: count, dtype: int64

Distribución de is_return (porcentaje):
is_return
False    97.849764
True      2.150236
Name: proportion, dtype: float64


In [ ]:
# Definimos qué entendemos por "registro duplicado exacto"
subset_cols = [
    "invoice",
    "stock_code",
    "description",
    "quantity",
    "price",
    "invoice_date",
    "customer_id",
    "country",
    "period"
]

# True = fila duplicada respecto a otra anterior
dup_mask = df_raw.duplicated(subset=subset_cols, keep="first")

# Nº total de duplicados
num_dups = dup_mask.sum()
perc_dups = num_dups / len(df_raw) * 100

print("Número de registros duplicados exactos:", num_dups)
print("Porcentaje sobre el total: {:.4f}%".format(perc_dups))

# Opcional: ver algunas filas duplicadas
print("\nEjemplo de duplicados (si existen):")
print(df_raw[dup_mask].head())


Número de registros duplicados exactos: 12133
Porcentaje sobre el total: 1.1367%

Ejemplo de duplicados (si existen):
    invoice stock_code                        description  quantity  \
371  489517      21912           VINTAGE SNAKES & LADDERS         1   
383  489517      22130    PARTY CONE CHRISTMAS DECORATION         6   
384  489517      22319  HAIRCLIPS FORTIES FABRIC ASSORTED        12   
385  489517      21913     VINTAGE SEASIDE JIGSAW PUZZLES         1   
386  489517      21821    GLITTER STAR GARLAND WITH BELLS         1   

           invoice_date  price  customer_id         country     period  \
371 2009-12-01 11:34:00   3.75      16329.0  United Kingdom  2009-2010   
383 2009-12-01 11:34:00   0.85      16329.0  United Kingdom  2009-2010   
384 2009-12-01 11:34:00   0.65      16329.0  United Kingdom  2009-2010   
385 2009-12-01 11:34:00   3.75      16329.0  United Kingdom  2009-2010   
386 2009-12-01 11:34:00   3.75      16329.0  United Kingdom  2009-2010   

     reven

In [ ]:
# Eliminamos los duplicados exactos definidos arriba
df_clean = df_raw.drop_duplicates(subset=subset_cols, keep="first").reset_index(drop=True)

print("Shape original df_raw:", df_raw.shape)
print("Shape limpio df_clean:", df_clean.shape)


Shape original df_raw: (1067371, 11)
Shape limpio df_clean: (1055238, 11)


In [ ]:
# Cuántas filas tienen / no tienen customer_id en df_clean
print("Nulos en customer_id:", df_clean["customer_id"].isna().sum())
print("Total filas df_clean:", len(df_clean))

print("\nDistribución (conteo):")
print(df_clean["customer_id"].isna().value_counts())

print("\nDistribución (porcentaje):")
print(df_clean["customer_id"].isna().value_counts(normalize=True) * 100)


Nulos en customer_id: 242870
Total filas df_clean: 1055238

Distribución (conteo):
customer_id
False    812368
True     242870
Name: count, dtype: int64

Distribución (porcentaje):
customer_id
False    76.984339
True     23.015661
Name: proportion, dtype: float64


In [ ]:
# Nos quedamos solo con las filas que sí tienen customer_id
df_segmentation = df_clean[df_clean["customer_id"].notna()].copy()

print("Shape df_clean (todas las transacciones limpias):", df_clean.shape)
print("Shape df_segmentation (solo con customer_id):", df_segmentation.shape)

# Nº de clientes distintos
num_clientes = df_segmentation["customer_id"].nunique()
print("Número de clientes distintos en df_segmentation:", num_clientes)


Shape df_clean (todas las transacciones limpias): (1055238, 11)
Shape df_segmentation (solo con customer_id): (812368, 11)
Número de clientes distintos en df_segmentation: 5942


In [ ]:
df_segmentation["customer_id"] = df_segmentation["customer_id"].astype(int)

print(df_segmentation[["customer_id"]].head())
print(df_segmentation["customer_id"].dtype)


   customer_id
0        13085
1        13085
2        13085
3        13085
4        13085
int64


In [ ]:
from google.colab import files

# Guardar en CSV
df_segmentation.to_csv("transacciones_segmentation.csv", index=False)
df_clean.to_csv("transacciones_clean_todas.csv", index=False)

# Descargar a tu ordenador
files.download("transacciones_segmentation.csv")
files.download("transacciones_clean_todas.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Fecha máxima de compra en df_segmentation
max_date = df_segmentation["invoice_date"].max()
snapshot_date = max_date + pd.Timedelta(days=1)

print("Fecha máxima de compra:", max_date)
print("Fecha de referencia (snapshot_date):", snapshot_date)


Fecha máxima de compra: 2011-12-09 12:50:00
Fecha de referencia (snapshot_date): 2011-12-10 12:50:00


In [ ]:
# Agrupamos por cliente y calculamos métricas de comportamiento
df_customers = (
    df_segmentation
      .groupby("customer_id")
      .agg(
          # Recency: días desde la última compra hasta snapshot_date
          last_purchase=("invoice_date", "max"),
          first_purchase=("invoice_date", "min"),
          n_invoices=("invoice", "nunique"),    # nº de facturas distintas (frecuencia de compra)
          n_lines=("invoice", "size"),          # nº de líneas de ticket (todas las posiciones compradas)
          quantity_total=("quantity", "sum"),   # unidades totales compradas
          revenue_total=("revenue", "sum"),     # Monetary bruto (ventas - devoluciones)
          n_products=("stock_code", "nunique")  # Variedad U: nº de referencias distintas
      )
      .reset_index()
)

# Ahora calculamos la Recency en días, a partir de last_purchase
df_customers["recency_days"] = (snapshot_date - df_customers["last_purchase"]).dt.days

# Frequency (F): nº de compras/facturas
df_customers["frequency"] = df_customers["n_invoices"]

# Monetary (M): revenue_total (podemos renombrar para claridad)
df_customers["monetary"] = df_customers["revenue_total"]

# Variedad U: nº de productos distintos
df_customers["variety_u"] = df_customers["n_products"]

print(df_customers.head())


   customer_id       last_purchase      first_purchase  n_invoices  n_lines  \
0        12346 2011-01-18 10:17:00 2009-12-14 08:34:00          17       47   
1        12347 2011-12-07 15:52:00 2010-10-31 14:20:00           8      253   
2        12348 2011-09-25 13:13:00 2010-09-27 14:59:00           5       51   
3        12349 2011-11-21 09:51:00 2009-12-04 12:49:00           5      180   
4        12350 2011-02-02 16:01:00 2011-02-02 16:01:00           1       17   

   quantity_total  revenue_total  n_products  recency_days  frequency  \
0              53         -51.74          30           326         17   
1            3286        5633.32         126             2          8   
2            2714        2019.40          25            75          5   
3            1619        4404.54         139            19          5   
4             197         334.40          17           310          1   

   monetary  variety_u  
0    -51.74         30  
1   5633.32        126  
2   2019.40

In [ ]:
print("Shape df_customers:", df_customers.shape)
print("\nDescripción estadística de recency, frequency, monetary y variety_u:")
print(df_customers[["recency_days", "frequency", "monetary", "variety_u"]].describe())


Shape df_customers: (5942, 12)

Descripción estadística de recency, frequency, monetary y variety_u:
       recency_days    frequency       monetary    variety_u
count   5942.000000  5942.000000    5942.000000  5942.000000
mean     202.908617     7.552339    2792.560250    81.324806
std      211.857936    15.972262   13971.075857   116.397649
min        1.000000     1.000000  -25111.090000     1.000000
25%       25.000000     2.000000     323.010000    18.000000
50%       96.000000     4.000000     840.720000    44.000000
75%      381.000000     8.000000    2172.230000   102.000000
max      739.000000   510.000000  598215.220000  2557.000000


In [ ]:
# Duración de la relación con la tienda (en días)
df_customers["customer_lifetime_days"] = (
    df_customers["last_purchase"] - df_customers["first_purchase"]
).dt.days + 1  # +1 para contar ambos extremos

print(df_customers[["customer_id", "first_purchase", "last_purchase", "customer_lifetime_days"]].head())


   customer_id      first_purchase       last_purchase  customer_lifetime_days
0        12346 2009-12-14 08:34:00 2011-01-18 10:17:00                     401
1        12347 2010-10-31 14:20:00 2011-12-07 15:52:00                     403
2        12348 2010-09-27 14:59:00 2011-09-25 13:13:00                     363
3        12349 2009-12-04 12:49:00 2011-11-21 09:51:00                     717
4        12350 2011-02-02 16:01:00 2011-02-02 16:01:00                       1


In [ ]:
# Ticket medio (Average Order Value): ingreso neto por factura
df_customers["avg_order_value"] = df_customers["monetary"] / df_customers["frequency"]

# Unidades medias por factura
df_customers["avg_items_per_invoice"] = df_customers["quantity_total"] / df_customers["frequency"]

# Líneas de ticket medias por factura
df_customers["avg_lines_per_invoice"] = df_customers["n_lines"] / df_customers["frequency"]

print(df_customers[[
    "customer_id", "frequency", "monetary",
    "avg_order_value", "avg_items_per_invoice", "avg_lines_per_invoice"
]].head())


   customer_id  frequency  monetary  avg_order_value  avg_items_per_invoice  \
0        12346         17    -51.74        -3.043529               3.117647   
1        12347          8   5633.32       704.165000             410.750000   
2        12348          5   2019.40       403.880000             542.800000   
3        12349          5   4404.54       880.908000             323.800000   
4        12350          1    334.40       334.400000             197.000000   

   avg_lines_per_invoice  
0               2.764706  
1              31.625000  
2              10.200000  
3              36.000000  
4              17.000000  


In [ ]:
# Revenue asociado a devoluciones y a compras "normales"
df_segmentation["return_revenue"] = df_segmentation["revenue"].where(df_segmentation["is_return"], 0)
df_segmentation["purchase_revenue"] = df_segmentation["revenue"].where(~df_segmentation["is_return"], 0)

print(df_segmentation[["invoice", "is_return", "revenue", "return_revenue", "purchase_revenue"]].head(10))


  invoice  is_return  revenue  return_revenue  purchase_revenue
0  489434      False     83.4             0.0              83.4
1  489434      False     81.0             0.0              81.0
2  489434      False     81.0             0.0              81.0
3  489434      False    100.8             0.0             100.8
4  489434      False     30.0             0.0              30.0
5  489434      False     39.6             0.0              39.6
6  489434      False     30.0             0.0              30.0
7  489434      False     59.5             0.0              59.5
8  489435      False     30.6             0.0              30.6
9  489435      False     45.0             0.0              45.0


In [ ]:
returns_info = (
    df_segmentation
      .groupby("customer_id")
      .agg(
          return_lines=("is_return", "sum"),           # nº de líneas de devolución
          total_lines=("is_return", "size"),           # nº total de líneas
          return_revenue=("return_revenue", "sum"),    # revenue total de devoluciones (negativo)
          purchase_revenue=("purchase_revenue", "sum") # revenue de compras "normales"
      )
      .reset_index()
)

# Ratios
returns_info["return_ratio_lines"] = returns_info["return_lines"] / returns_info["total_lines"]

# Para el revenue, usamos valor absoluto para interpretar mejor el peso de las devoluciones
returns_info["return_ratio_revenue"] = (
    returns_info["return_revenue"].abs() /
    (returns_info["purchase_revenue"].abs() + 1e-9)
)

print(returns_info.head())


   customer_id  return_lines  total_lines  return_revenue  purchase_revenue  \
0        12346            13           47       -77608.20          77556.46   
1        12347             0          253            0.00           5633.32   
2        12348             0           51            0.00           2019.40   
3        12349             5          180          -24.15           4428.69   
4        12350             0           17            0.00            334.40   

   return_ratio_lines  return_ratio_revenue  
0            0.276596              1.000667  
1            0.000000              0.000000  
2            0.000000              0.000000  
3            0.027778              0.005453  
4            0.000000              0.000000  


In [ ]:
df_customers = df_customers.merge(
    returns_info[[
        "customer_id",
        "return_lines",
        "total_lines",
        "return_revenue",
        "purchase_revenue",
        "return_ratio_lines",
        "return_ratio_revenue"
    ]],
    on="customer_id",
    how="left"
)

print(df_customers[[
    "customer_id", "n_lines", "return_lines",
    "return_ratio_lines", "monetary",
    "return_revenue", "purchase_revenue", "return_ratio_revenue"
]].head())


   customer_id  n_lines  return_lines  return_ratio_lines  monetary  \
0        12346       47            13            0.276596    -51.74   
1        12347      253             0            0.000000   5633.32   
2        12348       51             0            0.000000   2019.40   
3        12349      180             5            0.027778   4404.54   
4        12350       17             0            0.000000    334.40   

   return_revenue  purchase_revenue  return_ratio_revenue  
0       -77608.20          77556.46              1.000667  
1            0.00           5633.32              0.000000  
2            0.00           2019.40              0.000000  
3          -24.15           4428.69              0.005453  
4            0.00            334.40              0.000000  


In [ ]:
# Comprobacion
check_diff = (
    df_customers["purchase_revenue"] + df_customers["return_revenue"]
    - df_customers["monetary"]
).abs().max()

print("Máxima diferencia entre (purchase_revenue + return_revenue) y monetary:", check_diff)


Máxima diferencia entre (purchase_revenue + return_revenue) y monetary: 2.9103830456733704e-11


In [ ]:
print("Shape final df_customers:", df_customers.shape)

cols_resumen = [
    "recency_days", "frequency", "monetary", "variety_u",
    "customer_lifetime_days",
    "avg_order_value", "avg_items_per_invoice", "avg_lines_per_invoice",
    "return_ratio_lines", "return_ratio_revenue"
]

print("\nDescripción estadística de las métricas principales:")
print(df_customers[cols_resumen].describe())


Shape final df_customers: (5942, 22)

Descripción estadística de las métricas principales:
       recency_days    frequency       monetary    variety_u  \
count   5942.000000  5942.000000    5942.000000  5942.000000   
mean     202.908617     7.552339    2792.560250    81.324806   
std      211.857936    15.972262   13971.075857   116.397649   
min        1.000000     1.000000  -25111.090000     1.000000   
25%       25.000000     2.000000     323.010000    18.000000   
50%       96.000000     4.000000     840.720000    44.000000   
75%      381.000000     8.000000    2172.230000   102.000000   
max      739.000000   510.000000  598215.220000  2557.000000   

       customer_lifetime_days  avg_order_value  avg_items_per_invoice  \
count             5942.000000      5942.000000            5942.000000   
mean               276.396331       291.612052             204.203323   
std                259.688017       520.861772            1232.186079   
min                  1.000000    -25111.

In [ ]:
# Partimos de df_segmentation pero nos quedamos solo con compras "normales"
df_purchases = df_segmentation[~df_segmentation["is_return"]].copy()

# Agrupamos a nivel factura (invoice) por cliente
df_invoices = (
    df_purchases
      .groupby(["customer_id", "invoice"])
      .agg(
          invoice_date=("invoice_date", "max"),  # fecha de la factura
          revenue_invoice=("revenue", "sum")     # revenue neto de esa factura
      )
      .reset_index()
)

print("Shape df_invoices:", df_invoices.shape)
print(df_invoices.head())


Shape df_invoices: (36975, 4)
   customer_id invoice        invoice_date  revenue_invoice
0        12346  491725 2009-12-14 08:34:00             45.0
1        12346  491742 2009-12-14 11:00:00             22.5
2        12346  491744 2009-12-14 11:02:00             22.5
3        12346  492718 2009-12-18 10:47:00             22.5
4        12346  492722 2009-12-18 10:55:00              1.0


In [ ]:
def compute_interpurchase_stats(g):
    # g es el subconjunto de df_invoices de un cliente
    g = g.sort_values("invoice_date")
    diffs = g["invoice_date"].diff().dt.days.dropna()  # diferencias en días entre compras sucesivas

    if len(diffs) == 0:
        # Cliente con solo una compra: no hay diferencia entre compras
        return pd.Series({
            "avg_days_between_purchases": None,
            "std_days_between_purchases": None,
            "num_purchases_non_return": len(g)
        })
    else:
        return pd.Series({
            "avg_days_between_purchases": diffs.mean(),
            "std_days_between_purchases": diffs.std(),
            "num_purchases_non_return": len(g)
        })

interpurchase_stats = (
    df_invoices
      .groupby("customer_id")
      .apply(compute_interpurchase_stats)
      .reset_index()
)

print(interpurchase_stats.head())


   customer_id  avg_days_between_purchases  std_days_between_purchases  \
0        12346                   35.909091                   65.426989   
1        12347                   57.000000                   19.035055   
2        12348                   90.500000                   57.703264   
3        12349                  189.666667                  187.040994   
4        12350                         NaN                         NaN   

   num_purchases_non_return  
0                      12.0  
1                       8.0  
2                       5.0  
3                       4.0  
4                       1.0  


/tmp/ipython-input-2507482201.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_interpurchase_stats)


In [ ]:
# Día de la semana: Monday=0, Sunday=6
df_invoices["weekday"] = df_invoices["invoice_date"].dt.weekday
df_invoices["is_weekend"] = df_invoices["weekday"] >= 5  # sábado (5) o domingo (6)

weekend_stats = (
    df_invoices
      .groupby("customer_id")
      .agg(
          weekend_purchases=("is_weekend", "sum"),
          total_purchases_non_return=("is_weekend", "size")
      )
      .reset_index()
)

weekend_stats["weekend_ratio"] = (
    weekend_stats["weekend_purchases"] /
    weekend_stats["total_purchases_non_return"]
)

print(weekend_stats.head())


   customer_id  weekend_purchases  total_purchases_non_return  weekend_ratio
0        12346                  0                          12          0.000
1        12347                  1                           8          0.125
2        12348                  1                           5          0.200
3        12349                  0                           4          0.000
4        12350                  0                           1          0.000


In [ ]:
# Unimos periodicidad de compra
df_customers = df_customers.merge(
    interpurchase_stats,
    on="customer_id",
    how="left"
)

# Unimos hábitos de fin de semana
df_customers = df_customers.merge(
    weekend_stats[["customer_id", "weekend_purchases", "total_purchases_non_return", "weekend_ratio"]],
    on="customer_id",
    how="left"
)

print("Shape df_customers tras añadir hábitos temporales:", df_customers.shape)

cols_temporales = [
    "customer_id",
    "frequency",
    "num_purchases_non_return",
    "avg_days_between_purchases",
    "std_days_between_purchases",
    "weekend_purchases",
    "total_purchases_non_return",
    "weekend_ratio"
]

print(df_customers[cols_temporales].head())


Shape df_customers tras añadir hábitos temporales: (5942, 28)
   customer_id  frequency  num_purchases_non_return  \
0        12346         17                      12.0   
1        12347          8                       8.0   
2        12348          5                       5.0   
3        12349          5                       4.0   
4        12350          1                       1.0   

   avg_days_between_purchases  std_days_between_purchases  weekend_purchases  \
0                   35.909091                   65.426989                0.0   
1                   57.000000                   19.035055                1.0   
2                   90.500000                   57.703264                1.0   
3                  189.666667                  187.040994                0.0   
4                         NaN                         NaN                0.0   

   total_purchases_non_return  weekend_ratio  
0                        12.0          0.000  
1                         8.0   